In [3]:
path=r'C:/Users/lenovo/Desktop/[DM] Group Project/Data-Mining-Project-master/dataset/'

In [1]:
import pandas as pd
import numpy as np

In [4]:
train = pd.read_csv(path+"train.csv")
test = pd.read_csv(path+"test.csv")
labels = pd.read_csv(path+"test_labels.csv")

In [5]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:
train.isnull().any(),test.isnull().any()

(id               False
 comment_text     False
 toxic            False
 severe_toxic     False
 obscene          False
 threat           False
 insult           False
 identity_hate    False
 dtype: bool, id              False
 comment_text    False
 dtype: bool)

In [7]:
classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[classes].values
list_sentences_train = train["comment_text"]
list_sentences_test = test["comment_text"]

In [13]:
from keras.preprocessing.text import Tokenizer

In [14]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

In [15]:
from keras.preprocessing.sequence import pad_sequences    #用于填充序列为maxlen：keras只能接受长度相同的序列输入。因此如果目前序列长度参差不齐，这时需要使用pad_sequences()。该函数是将序列转化为经过填充以后的一个长度相同的新序列新序列。

In [16]:
maxlen = 200
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)    #train set
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)    #test set

In [17]:
gl_path = r'C:/Users/lenovo/Desktop/[DM] Group Project/glove.twitter.27B.25d.txt'
ft_path = r'C:/Users/lenovo/Desktop/[DM] Group Project/wiki.simple.vec'
wv_path = r'C:/Users/lenovo/Desktop/[DM] Group Project/model.bin'

In [19]:
import gensim.models.keyedvectors as word2vec
import gc

In [27]:
def loadEmbeddingMatrix(typeToLoad):    #to load all 3 types of embedding matrix
        if(typeToLoad=="glove"):
            EMBEDDING_FILE=gl_path
            embed_size = 25
        elif(typeToLoad=="word2vec"):
            word2vecDict = word2vec.KeyedVectors.load_word2vec_format(wv_path, binary=True)
            embed_size = 300
        elif(typeToLoad=="fasttext"):
            EMBEDDING_FILE=ft_path
            embed_size = 300

        if(typeToLoad=="glove" or typeToLoad=="fasttext" ):
            embeddings_index = dict()
            f = open(EMBEDDING_FILE, encoding='gb18030', errors='ignore')
            for line in f:
                values = line.split()
                word = values[0]
                coefs = np.asarray(values[1:], dtype='float32')            
                if len(coefs) != 25:
                    continue
                embeddings_index[word] = coefs
                
            f.close()
            print('Loaded %s word vectors.' % len(embeddings_index))
        else:
            embeddings_index = dict()
            for word in word2vecDict.wv.vocab:
                embeddings_index[word] = word2vecDict.word_vec(word)
            print('Loaded %s word vectors.' % len(embeddings_index))
            
        gc.collect()
        all_embs = np.stack(list(embeddings_index.values()))
        emb_mean,emb_std = all_embs.mean(), all_embs.std()
        
        nb_words = len(tokenizer.word_index)
        embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
        gc.collect()

        embeddedCount = 0
        for word, i in tokenizer.word_index.items():
            i-=1
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None: 
                embedding_matrix[i] = embedding_vector
                embeddedCount+=1
        print('total embedded:',embeddedCount,'common words')
        
        del(embeddings_index)
        gc.collect()
        
        return embedding_matrix

In [55]:
embedding_matrix = loadEmbeddingMatrix('glove')    #load 'glove' embedding matrix

Loaded 1175734 word vectors.
total embedded: 79113 common words


In [30]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D,Bidirectional
from keras.models import Model

In [31]:
inp = Input(shape=(maxlen, ))#Define a keras Input tensor  #Start here, configure the model

In [33]:
x = Embedding(len(tokenizer.word_index), embedding_matrix.shape[1],weights=[embedding_matrix],trainable=False)(inp)    #Convert a positive integer (index value) to a dense vector of fixed size
x = Bidirectional(LSTM(60, return_sequences=True,name='lstm_layer',dropout=0.1,recurrent_dropout=0.1))(x)    #Realize the bidirectional structure of RNN type neural network

In [34]:
#config the output of the model:
x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)

In [35]:
import keras.metrics as metrics    #Neural Network Evaluation Methods

In [36]:
model = Model(inputs=inp, outputs=x)    #Define the model here (have not entered the training set for training yet) (end configuring the model)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=[    #Use MSE and AUC as evaluation indicators
                          metrics.MeanSquaredError(),
                          metrics.AUC(),
                  ])

In [37]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 200, 25)           5258425   
                                                                 
 bidirectional_1 (Bidirectio  (None, 200, 120)         41280     
 nal)                                                            
                                                                 
 global_max_pooling1d (Globa  (None, 120)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 120)               0         
                                                                 
 dense (Dense)               (None, 50)                6050  

In [38]:
from sklearn.model_selection import train_test_split    #spilt train and validation set
X_train, X_val, y_train, y_val = train_test_split(X_t,y,test_size=0.2,random_state=1)     #use the x,y of the original train

In [39]:
batch_size = 128
epochs = 2
hist = model.fit(X_train,y_train, batch_size=batch_size, epochs=epochs)    #train the model using the train set (x and y)

Epoch 1/2
998/998 [==============================] - 903s 901ms/step - loss: 0.1326 - mean_squared_error: 0.0333 - auc: 0.8169
Epoch 2/2
998/998 [==============================] - 926s 928ms/step - loss: 0.1034 - mean_squared_error: 0.0267 - auc: 0.9002


In [40]:
preds_train = model.predict(X_train)

In [53]:
preds_train

array([[1.3623595e-02, 3.2344460e-04, 4.4855177e-03, 1.6424060e-04,
        3.4399033e-03, 7.5829029e-04],
       [1.2687048e-01, 7.4063241e-03, 4.7938168e-02, 4.2207539e-03,
        4.8898876e-02, 9.8959208e-03],
       [2.3175627e-02, 6.0126185e-04, 6.1507821e-03, 3.2186508e-04,
        6.3243210e-03, 1.5035570e-03],
       ...,
       [8.4456414e-02, 1.0075003e-02, 4.3412924e-02, 4.4140816e-03,
        3.7735641e-02, 1.2299478e-02],
       [6.2018305e-02, 5.3495765e-03, 2.9902428e-02, 2.2993088e-03,
        2.4940401e-02, 7.1427524e-03],
       [3.0706561e-01, 3.6669910e-02, 1.7276996e-01, 9.9048018e-03,
        1.7223799e-01, 2.0926386e-02]], dtype=float32)

In [41]:
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score

print(roc_auc_score(y_train, preds_train))

0.908518351978887


In [42]:
preds_val = model.predict(X_val)
print(roc_auc_score(y_val, preds_val))

0.8961344328164914


In [59]:
preds_val

array([[5.1290977e-01, 8.0278933e-02, 3.0935419e-01, 2.8832287e-02,
        3.5673428e-01, 4.9698800e-02],
       [1.1185366e-01, 2.6380420e-03, 3.7509620e-02, 1.7556846e-03,
        3.7862450e-02, 6.6973865e-03],
       [2.2268474e-02, 5.0884485e-04, 6.5262318e-03, 2.7891994e-04,
        5.3603649e-03, 1.1652410e-03],
       ...,
       [7.6115519e-02, 7.2779655e-03, 3.8205326e-02, 3.4002960e-03,
        2.9074788e-02, 1.0733515e-02],
       [5.7007879e-02, 5.3457320e-03, 2.6141226e-02, 2.0368695e-03,
        2.1537781e-02, 6.8677366e-03],
       [2.9756570e-01, 2.3902565e-02, 1.3929316e-01, 1.1374146e-02,
        1.5674523e-01, 2.4024814e-02]], dtype=float32)

In [43]:
labels = labels[classes]
sum_labels=np.sum(labels.values,axis=1)
# print(sum_labels)
idx=sum_labels>=0
y_test = labels[idx]
X_test = X_te[idx]

In [44]:
preds_test = model.predict(X_test)    #predict using the test's x

In [56]:
print(roc_auc_score(y_test, preds_test))    #test accuracy for 'glove'

0.8731585814602864
